In [ ]:
#only necessary when running in Colab
#Remove the "#" symbols from the beginning of the 4 lines that start with !
#You only want to run this once per session, so then put them back before you "restart and run all at the end of a game"
#!apt-get remove pandoc
#!wget https://github.com/jgm/pandoc/releases/download/2.18/pandoc-2.18-1-amd64.deb
#!dpkg -i  pandoc-2.18-1-amd64.deb
#!pip install wikipedia pypandoc

In [ ]:
import wikipedia
import ipywidgets
import re
import textwrap
import urllib
import pypandoc
from IPython.display import display,Markdown

In [ ]:
level = ipywidgets.IntSlider(
    value=4,
    min=1,
    max=5,
    step=1,
    description='Difficulty')

level

In [ ]:
urlname = f'https://randomincategory.toolforge.org/?category=All%20Wikipedia%20level-{level.value}%20vital%20articles&server=en.wikipedia.org&cmnamespace=&cmtype=&returntype=subject'

try:
    name = urllib.request.urlopen(urlname).read(500).decode().split('title>')[1].split(' - Wikipedia')[0]
    content = wikipedia.page(name,auto_suggest=False).content
except:
    name = urllib.request.urlopen(urlname).read(500).decode().split('title>')[1].split(' - Wikipedia')[0]
    content = wikipedia.page(name,auto_suggest=False).content

def print_redacted(new_string,count_to_word_dict,bold=False):
    string_to_print = ''
    delims=['\n',r'===',r'==',r'.',r',',r'(',r')',r'[',r']',r'<',r'>',r'"',r':',r';',r'-',r'/',r'–',r'=',r'%',' ']
    for word in re.split(r'([ |\n|=|.|,|(|)|[|]|<|>|"|:|;|-|/|–|%])',new_string):
        if type(word)==str:
            if word in delims:
                string_to_print += word
            elif word in count_to_word_dict.keys():
                if len(count_to_word_dict[word])>0:
                    string_to_print += count_to_word_dict[word][0]
            else:
                pass
    if bold:
        print(string_to_print)
        string_to_print = pypandoc.convert_text(string_to_print,'markdown_strict',format='mediawiki',extra_args=["--shift-heading-level-by=1","--markdown-headings=atx"])
        string_to_print = re.sub('(?<=[A-z])(\n)(?=[A-z])',' ',string_to_print) # remove random extra newlines
        string_to_print = re.sub('(#{3,6} )(?P<Header>.*)(?=\n\n)','**\g<Header>** ',string_to_print) #make all headers just bold
    else:
        display(Markdown(string_to_print))
    
def create_replaced(input_string,word_to_count_dict,count_to_word_dict,count_the_word_dict,count):
    delims=['\n',r'.',r',',r'(',r')',r'[',r']',r'<',r'>',r'"',r':',r';',r'-',r'/',r'–',r'=',r'%',r'',r' ']
    words = [x for x in re.split(r'([ |\n|=|.|,|(|)|[|]|<|>|"|:|;|-|/|–|%])',input_string)]
    new_string = ''

    for word in words:
        if type(word)==str:
            word_lower = word.lower()
        if word in delims:
            new_string += word
        elif word_lower not in word_to_count_dict.keys():
            word_to_count_dict[word_lower]={word:str(count)}
            count_to_word_dict[str(count)]=['-'*len(word),word]
            new_string += str(count)
            count += 1
            count_the_word_dict[word_lower]=1
        else:
            if word not in word_to_count_dict[word_lower].keys():
                word_to_count_dict[word_lower][word]=str(count)
                count_to_word_dict[str(count)]=['-'*len(word),word]
                new_string += str(count)
                count += 1
                count_the_word_dict[word_lower]+=1
            else:
                new_string += word_to_count_dict[word_lower][word]
                count_the_word_dict[word_lower]+=1
    no_delim_lowercase_words = [x.lower() for x in words if x not in delims]
    return new_string,no_delim_lowercase_words,word_to_count_dict,count_to_word_dict,count_the_word_dict,count
    

word_to_count_dict = {}
count_to_word_dict = {}
count_the_word_dict = {}
count = 9897098660698

content_string,_,word_to_count_dict,count_to_word_dict,count_the_word_dict,count = create_replaced(content,word_to_count_dict,count_to_word_dict,count_the_word_dict,count)
name_string,name_list,word_to_count_dict,count_to_word_dict,count_the_word_dict,count = create_replaced(name,word_to_count_dict,count_to_word_dict,count_the_word_dict,count)

tried = []

starting_guesses = ['a','an','is','the','of', 'in', 'to', 'was', 'for', 'as',
'around', 'from', 'since', 'with', 'and', 'or', 'on', 'along', 'through', 'amid', 
'into', 'like', 'by', 'between', 'than', 'about', 'until', 'at', 'but', 'among', 
'following', 'over', 'after', 'during', 'behind', 'up', 'down', 'before', 'onto', 
'without', 'unlike', 'because', 'against', 'inside', 'outside','except','per']

for eachguess in starting_guesses:
    try:
        tried.append(eachguess)
        count = word_to_count_dict[eachguess]
        for eachcap in word_to_count_dict[eachguess].keys():
            count = word_to_count_dict[eachguess][eachcap]
            count_to_word_dict[count]=count_to_word_dict[count][1:]
        name_list.remove(eachguess)
    except:
        pass
    
#guess = ipywidgets.Text(value='the')
#guess
#thisguess = guess.value.lower()

def execute_guess(name_string,content_string,count_to_word_dict,count_the_word_dict,word_to_count_dict,thisguess):
    thisguess_lower = thisguess.lower()
    if thisguess_lower in tried:
        display(Markdown('Already tried'))
        print_redacted(name_string,count_to_word_dict,bold=True)
        print_redacted(content_string,count_to_word_dict)
    elif thisguess_lower not in word_to_count_dict.keys():
        display(Markdown('Nope'))
        print_redacted(name_string,count_to_word_dict,bold=True)
        print_redacted(content_string,count_to_word_dict)
        tried.append(thisguess_lower)
    else:
        tried.append(thisguess_lower)
        if thisguess_lower in name_list:
            name_list.remove(thisguess_lower)
        if len(name_list)>0:
            for eachcap in word_to_count_dict[thisguess_lower].keys():
                count = word_to_count_dict[thisguess_lower][eachcap]
                count_to_word_dict[count]=count_to_word_dict[count][1:]
            display(Markdown(f"{thisguess} appears {count_the_word_dict[thisguess_lower]} times"))
        else:
            display(Markdown('Hooray! \n'))
            for k,v in count_to_word_dict.items():
                if len(v)>1:
                    count_to_word_dict[k]=v[1:]
        print_redacted(name_string,count_to_word_dict,bold=True)
        print_redacted(content_string,count_to_word_dict)

In [ ]:
thisguess = "the"
execute_guess(name_string,content_string,count_to_word_dict,count_the_word_dict,word_to_count_dict,thisguess)

# Answers/Give up?

In [ ]:
#Give up? Uncomment(remove the # from) the line below and run 
#print(name)